In [126]:
import sage.libs.lrcalc.lrcalc as lrcalc
import itertools

#the function BL_vertices([n1,n2,...nt,n]) lists the vertices for 
#generic ni x n B_i's.
#BL_inequalities does the same but for inequalities.

#to test it, run this cell and then the next one.


def add_schurs(list1, list2):
    for x in list2.keys():
        if x in list1.keys():
            list1[x] = list1[x]+list2[x]
        else:
            list1.update({x:list2[x]})

def multiply_schurs(list1, list2, maxrows):
    list3 = {}
    for x in list1.keys():
        for y in list2.keys():
            z = lrcalc.mult(x,y,maxrows)
            add_schurs(list3, z)
    return list3
    
def multiply_many_schurs(poly_list, maxrows):
    p = lrcalc.mult([],[])
    for x in poly_list:
        p = multiply_schurs(p,x,maxrows)
    return p

def dominated(biglist, candidate):
    for listy in biglist:
        dom = true
        for i in range(len(candidate)-1):
            if candidate[i] > listy[i]:
                dom = false
        if candidate[-1] < listy[-1]:
            dom = false
        if dom == true:
            return true
    return false

def numbers_to_partition(bigdim, smalldim, subspace, intersection):
    partition = [bigdim + intersection - smalldim - subspace for i in range(intersection)]
    #partition.extend([0 for i in range(subspace - intersection)])
    return partition
    
def valid(schur,maxcols):
    schurs = [key for key,value in schur.items() if value ==1]
    #print schurs
    if len(schurs)==0:
        return false
    for k in schurs:
        if len(k) == 0:
            return true
    l = [max(partition) for partition in schurs]
    if min(l)<=maxcols:
        return true
    else:
        return false

def is_inequality(bigdim,smalldim_vector, subspace,intersection_vector):
    partitions = [lrcalc.mult([], numbers_to_partition(bigdim, smalldim_vector[i],subspace, intersection_vector[i])) for i in range(len(smalldim_vector))]
    #print partitions
    z = multiply_many_schurs(partitions, subspace)
    #print z
    return valid(z,bigdim - subspace)

def inequalities(dimension_vector):
    dimrange = list(range(1,dimension_vector[-1]))
    allrange = [list(range(i + 1)) for i in dimension_vector[0:-1]]
    for i in allrange:
        i.reverse()
    allrange.append(dimrange)
    potentials = cartesian_product_iterator(allrange)
    inequalities = []
    bigdim = dimension_vector[-1]

    smalldim_vector = dimension_vector[0:-1]

    for potential_inequality in potentials:
        if not dominated(inequalities,potential_inequality):
            #print potential_inequality
            subspace = potential_inequality[-1]
            intersection_vector = potential_inequality[0:-1]
            if is_inequality(bigdim, smalldim_vector, subspace, intersection_vector):
                inequalities.append(potential_inequality)
    return inequalities
    
def positive(m):
    positive = []
    for i in range(m):
        ineq = [0]
        ineq.extend([0 for j in range(i)])
        ineq.append(1)
        ineq.extend([0 for j in range(m - i -1)])
        positive.append(ineq)
    return positive

def BL_vertices(dimensionvector):
    ineqs = inequalities(dimensionvector)
    #print ineqs
    faces = [[ineq[-1]]+[-i for i in ineq[0:-1]] for ineq in ineqs]
    faces.extend(positive(len(dimensionvector)-1))
    p = Polyhedron(ieqs=faces, eqns=[ [dimensionvector[-1]] + [-i for i in dimensionvector[0:-1]] ])
    return p.Vrepresentation()

def P_vertices(dimensionvector):
    ineqs = inequalities(dimensionvector)
    #print(ineqs)
    faces = [[ineq[-1]]+[-i for i in ineq[0:-1]] for ineq in ineqs]
    faces.extend(positive(len(dimensionvector)-1))
    #print(faces)
    full = [dimensionvector[-1]] + [-i for i in dimensionvector[0:-1]]
    faces.append(full)
    #print(faces)
    p = Polyhedron(ieqs=faces)
    return p.Vrepresentation()

#maximizes sum n_i x_i over P, then outputs the face on which it is maximized. 
def P_slice(dimensionvector):
    ineqs = inequalities(dimensionvector)
    #print(ineqs)
    faces = [[ineq[-1]]+[-i for i in ineq[0:-1]] for ineq in ineqs]
    faces.extend(positive(len(dimensionvector)-1))
    #print(faces)
    full = [dimensionvector[-1]] + [-i for i in dimensionvector[0:-1]]
    faces.append(full)
    #print(faces)
    p = Polyhedron(ieqs=faces)
    p.to_linear_program
    lp, x = p.to_linear_program(return_variable=True)
    lp.set_objective(sum(dimensionvector[i]*x[i] for i in range(len(dimensionvector)-1)))
    k = lp.solve()
    print("unweighted maximum on P: ", k)
    newfull = [k] + [-i for i in dimensionvector[0:-1]]
    p = Polyhedron(ieqs=faces, eqns=[newfull])
    #print(lp)
    return p.Vrepresentation()



In [2]:
#example
BL_vertices([3,3,3,2,5])

(A vertex at (0, 0, 1, 1),
 A vertex at (0, 1, 0, 1),
 A vertex at (1, 0, 0, 1),
 A vertex at (1/2, 1/2, 1/2, 1/4))

In [3]:
#want to test if a vertex satisfies all inequalities but the sum one.
#so the question is - if a vertex satisfies all inequalities but sums to something else, is this the same as 
#let's get an empty polytope. 

BL_vertices([2,2,3])

()

In [47]:
P_vertices([2,2,2,2,7])

(A vertex at (0, 1, 1, 1),
 A vertex at (1, 0, 1, 1),
 A vertex at (1, 1, 0, 1),
 A vertex at (1, 1, 1, 0),
 A vertex at (1, 0, 1, 0),
 A vertex at (1, 1, 0, 0),
 A vertex at (1, 0, 0, 0),
 A vertex at (1, 0, 0, 1),
 A vertex at (0, 0, 0, 1),
 A vertex at (0, 0, 1, 1),
 A vertex at (0, 1, 0, 1),
 A vertex at (0, 1, 1, 0),
 A vertex at (0, 0, 1, 0),
 A vertex at (0, 1, 0, 0),
 A vertex at (0, 0, 0, 0))

In [63]:
BL_vertices([2,3,3,4])

(A vertex at (1/2, 1/2, 1/2),)

In [62]:
P_slice([2,3,3,6])

4


(A vertex at (1/2, 0, 1),
 A vertex at (1/2, 1, 0),
 A vertex at (1, 2/3, 0),
 A vertex at (1, 0, 2/3),
 A vertex at (0, 1, 1/3),
 A vertex at (0, 1/3, 1))

In [67]:
BL_vertices([1,2,2,4])

(A vertex at (0, 1, 1),)

In [105]:
#Oh no! 
P_slice([3,3,4,8])

unweighted maximum on P:  7


(A vertex at (0, 1, 1), A vertex at (1, 0, 1))

In [104]:
BL_vertices([3,3,4,8])

()

In [106]:
BL_vertices([3,3,4,7])

(A vertex at (0, 1, 1), A vertex at (1, 0, 1))

In [78]:
BL_vertices([2,2,2,3])

(A vertex at (1/2, 1/2, 1/2),)

In [79]:
BL_vertices([2,2,2,5])

()

In [88]:
P_slice([2,2,2,2,7])

unweighted maximum on P:  6


(A vertex at (0, 1, 1, 1),
 A vertex at (1, 1, 0, 1),
 A vertex at (1, 1, 1, 0),
 A vertex at (1, 0, 1, 1))

In [81]:
BL_vertices([2,2,2,4])

(A vertex at (0, 1, 1), A vertex at (1, 1, 0), A vertex at (1, 0, 1))

In [89]:
P_slice([2,2,2,5])

unweighted maximum on P:  4


(A vertex at (0, 1, 1), A vertex at (1, 1, 0), A vertex at (1, 0, 1))

In [93]:
BL_vertices([2,2,2,2,2,1])

(A vertex at (1/2, 0, 0, 0, 0),
 A vertex at (0, 1/2, 0, 0, 0),
 A vertex at (0, 0, 1/2, 0, 0),
 A vertex at (0, 0, 0, 0, 1/2),
 A vertex at (0, 0, 0, 1/2, 0))

In [109]:
BL_vertices([3,3,4,9])

()

In [111]:
P_slice([3,3,4,9])

unweighted maximum on P:  7


(A vertex at (0, 1, 1), A vertex at (1, 0, 1))

In [113]:
BL_vertices([3,3,4,7])

(A vertex at (0, 1, 1), A vertex at (1, 0, 1))

In [123]:
P_slice([2,3,3,4,10])

unweighted maximum on P:  10


(A vertex at (0, 1, 1, 1),)

In [124]:
BL_vertices([2,3,3,4,10])

(A vertex at (0, 1, 1, 1),)

In [135]:
#the maximum over P does not need to be integral.
P_slice([4,4,5,7])

unweighted maximum on P:  13/2


(A vertex at (1/2, 1/2, 1/2),)

In [134]:
BL_vertices([5,5,5,10])

(A vertex at (0, 1, 1), A vertex at (1, 1, 0), A vertex at (1, 0, 1))

# How about scaling to something less than it??

In [136]:
#for examples like e.g. [4,4,5,7] we had a non-integral maximum. Do I assume that comes from a moment polytope slice corresponding to a smaller operator?

In [137]:
#for just matrices - if you maximize over the flow polytope, will the maxima be flow polytopes for capacities that are as concentrated as possible?